# Standard Nystrom-based Spectral Clustering

This algorithm is from "[Fast Spectral Clustering via the Nystrom Method](http://www.cs.columbia.edu/~jebara/papers/ALT2013FSCVTNM.pdf)" by Choromanska et el.

In [6]:
from itertools import permutations
import numpy as np
import scipy as sp
import scipy.sparse.linalg
import scipy.cluster.vq

In [607]:
A = np.loadtxt('../data/processed/usps.csv', delimiter=',')

In [712]:
# np.random.shuffle(A)
inds = A[:, -1] < 3
X = A[inds, :-2]
Y = A[inds, -1].astype(int)
k = len(np.unique(Y))
n, d = X.shape
n, d

(2199, 254)

In [766]:
m = 500
# inds = np.random.choice(n, m, replace=False)
inds = np.arange(m)
inv_inds = np.array([a for a in range(n) if a not in inds])

In [767]:
A_11 = np.empty((m,m))
for i in range(m):
    for j in range(i, m):
        val = np.e ** (-1 * np.linalg.norm(X[inds[i]] - X[inds[j]]) ** 2)
        A_11[i, j] = val
        A_11[j, i] = val

In [768]:
C = np.empty((n-m, m))
for i in range(n-m):
    for j in range(m):
        val = np.e ** (-1 * np.linalg.norm(X[inv_inds[i]] - X[inds[j]]) ** 2)
        C[i, j] = val
C = np.vstack((A_11, C))

In [769]:
dd_hat = C.dot(np.linalg.inv(A_11)).dot(C.T).dot(np.ones(n))
D_hat = np.diag(dd_hat)
D_hat_inv = np.diag(1 / np.sqrt(dd_hat))

D_n1 = np.diag(C.dot(np.ones(m)))
D_11_inv = np.diag(1 / np.sqrt(A_11.dot(np.ones(m))))

In [770]:
M_hat_n1 = D_hat_inv.dot(C).dot(D_11_inv)

In [771]:
M_hat_n1.shape

(2199, 500)

In [772]:
M_11 = M_hat_n1[inds, :]
M_21 = M_hat_n1[inv_inds, :]

V, U = np.linalg.eig(np.linalg.inv(M_11))
# print(U)
M_11_inv = U.dot(np.diag(np.sqrt(V))).dot(U.T)

S = M_11 + M_11_inv.dot(M_21.T).dot(M_21).dot(M_11_inv)
U, L, T = np.linalg.svd(S)

V = M_hat_n1.dot(M_11_inv).dot(U).dot(np.diag(1 / np.sqrt(L)))

In [773]:
M_21.shape

(1699, 500)

In [774]:
Z = V[:, :k]
centroids, distortion = sp.cluster.vq.kmeans(Z, k)
centroids, distortion

(array([[-0.0356231 , -0.0024062 ],
        [-0.00148436, -0.00097147]]), 0.013008144022991702)

In [775]:
y_hat = np.zeros(n, dtype=int)
for i in range(n):
    dists = np.array([np.linalg.norm(Z[i] - c) for c in centroids])
    y_hat[i] = np.argmin(dists)

perms = []
for p in permutations(np.arange(1, k+1)):
    P = dict()
    for i in range(k):
        P[i] = p[i]
    perms.append(P)

accuracy = np.zeros(len(perms))
for i in range(len(perms)):
    yy = y_hat.copy()
    for key, val in perms[i].items():
        yy[y_hat == key] = val
    accuracy[i] = (Y == yy).sum() / n * 100
accuracy.max()

83.53797180536607

In [776]:
np.bincount(yy)

array([   0, 1556,  643], dtype=int64)

In [765]:
np.bincount(Y)

array([   0, 1194, 1005], dtype=int64)